# Cosmic Variance

In [1]:
import astropy.io.fits as fits
import numpy as np
from scipy.integrate import quad
import scipy.integrate as integrate
from scipy.optimize import curve_fit
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table,join
import os
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import treecorr
#import arviz as az
#import pandas as pd
#import pymc as pm
%matplotlib inline

import halomod as hm
import hmf


/home/astro/manasoi7/.conda/envs/project/lib/python3.12/site-packages/halomod/halo_model.py:32: UserWarning: Warning: Some Halo-Exclusion models have significant speedup when using Numba
  from .halo_exclusion import NoExclusion


In [2]:
cat_files = ["COSMOS_masked_cat.fits", "SN-C3_masked_cat.fits", 
             "SN-X3_masked_cat.fits", "SN-E2_masked_cat.fits"]
zm_files = ["Y3_deep_fields_DB_wKNN_cat_COSMOS_zm.fits", "Y3_deep_fields_DB_wKNN_cat_SN-C3_zm.fits", 
           "Y3_deep_fields_DB_wKNN_cat_SN-X3_zm.fits", "Y3_deep_fields_DB_wKNN_cat_SN-E2_zm.fits"]
rand_files = ["COSMOS_randoms_ugriz_trim_video.fits", "SN-C3_randoms_ugriz_trim_video.fits",
             "SN-X3_randoms_ugriz_trim_video.fits", "SN-E2_randoms_ugriz_trim_video.fits"]
names = ['COSMOS', 'SN-C3', 'SN-X3', 'SN-E2']

In [3]:
home_dir = os.path.expanduser('~')
# Construct the path to the "Thesis" directory on the desktop
thesis_path = os.path.join(home_dir, 'Master_Thesis', 'DATA')



# My catalog
fits_file_path = os.path.join(thesis_path, "Y3_deep_fields_DB_wKNN_cat_SN-E2_zm.fits")  # Replace with your actual file name
t= Table.read(fits_file_path)
masked = os.path.join(thesis_path, "SN-E2_masked_cat.fits")  # Replace with your actual file name

t3= Table.read(masked)


t=join(t,t3,keys='id')


t.rename_column('ra_1','ra')
t.rename_column('dec_1','dec')



#Randoms
fits_random = os.path.join(thesis_path, "SN-E2_randoms_ugriz_trim_video.fits") 

# Open the FITS file using astropy.io.fits
hdulist = fits.open(fits_random)
hdulist.info()

t2= Table.read(fits_random)

Filename: /home/astro/manasoi7/Master_Thesis/DATA/SN-E2_randoms_ugriz_trim_video.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  SN-E2_C01_randoms_raw.fits    1 BinTableHDU     18   11025433R x 3C   [E, E, 9A]   


In [4]:
t

id,ra,dec,fof_id_1,fof_size_1,mask_flags_1,flags_1,flagstr_1,badpix_frac_1,psf_g_0_1,psf_g_1_1,psf_T_1,bdf_T_1,bdf_T_err_1,bdf_T_ratio_1,bdf_g_0_1,bdf_g_1_1,bdf_fracdev_1,bdf_fracdev_err_1,tilename_1,EBV_SFD98_1,mask_flags_nir_1,flags_nir_1,flagstr_nir_1,badpix_frac_nir_1,psf_flux_u_1,psf_flux_g_1,psf_flux_r_1,psf_flux_i_1,psf_flux_z_1,psf_flux_J_1,psf_flux_H_1,psf_flux_Ks_1,psf_flux_err_u_1,psf_flux_err_g_1,psf_flux_err_r_1,psf_flux_err_i_1,psf_flux_err_z_1,psf_flux_err_J_1,psf_flux_err_H_1,psf_flux_err_Ks_1,bdf_flux_u_1,bdf_flux_g_1,bdf_flux_r_1,bdf_flux_i_1,bdf_flux_z_1,bdf_flux_J_1,bdf_flux_H_1,bdf_flux_Ks_1,bdf_flux_err_u_1,bdf_flux_err_g_1,bdf_flux_err_r_1,bdf_flux_err_i_1,bdf_flux_err_z_1,bdf_flux_err_J_1,bdf_flux_err_H_1,bdf_flux_err_Ks_1,gap_flux_u_1,gap_flux_g_1,gap_flux_r_1,gap_flux_i_1,gap_flux_z_1,gap_flux_J_1,gap_flux_H_1,gap_flux_Ks_1,gap_flux_err_u_1,gap_flux_err_g_1,gap_flux_err_r_1,gap_flux_err_i_1,gap_flux_err_z_1,gap_flux_err_J_1,gap_flux_err_H_1,gap_flux_err_Ks_1,psf_flux_calib_u_1,psf_flux_calib_g_1,psf_flux_calib_r_1,psf_flux_calib_i_1,psf_flux_calib_z_1,psf_flux_calib_J_1,psf_flux_calib_H_1,psf_flux_calib_Ks_1,psf_flux_err_calib_u_1,psf_flux_err_calib_g_1,psf_flux_err_calib_r_1,psf_flux_err_calib_i_1,psf_flux_err_calib_z_1,psf_flux_err_calib_J_1,psf_flux_err_calib_H_1,psf_flux_err_calib_Ks_1,bdf_flux_calib_u_1,bdf_flux_calib_g_1,bdf_flux_calib_r_1,bdf_flux_calib_i_1,bdf_flux_calib_z_1,bdf_flux_calib_J_1,bdf_flux_calib_H_1,bdf_flux_calib_Ks_1,bdf_flux_err_calib_u_1,bdf_flux_err_calib_g_1,bdf_flux_err_calib_r_1,bdf_flux_err_calib_i_1,bdf_flux_err_calib_z_1,bdf_flux_err_calib_J_1,bdf_flux_err_calib_H_1,bdf_flux_err_calib_Ks_1,psf_flux_dered_u_1,psf_flux_dered_g_1,psf_flux_dered_r_1,psf_flux_dered_i_1,psf_flux_dered_z_1,psf_flux_dered_J_1,psf_flux_dered_H_1,psf_flux_dered_Ks_1,psf_flux_err_dered_u_1,psf_flux_err_dered_g_1,psf_flux_err_dered_r_1,psf_flux_err_dered_i_1,psf_flux_err_dered_z_1,psf_flux_err_dered_J_1,psf_flux_err_dered_H_1,psf_flux_err_dered_Ks_1,bdf_flux_dered_u_1,bdf_flux_dered_g_1,bdf_flux_dered_r_1,bdf_flux_dered_i_1,bdf_flux_dered_z_1,bdf_flux_dered_J_1,bdf_flux_dered_H_1,bdf_flux_dered_Ks_1,bdf_flux_err_dered_u_1,bdf_flux_err_dered_g_1,bdf_flux_err_dered_r_1,bdf_flux_err_dered_i_1,bdf_flux_err_dered_z_1,bdf_flux_err_dered_J_1,bdf_flux_err_dered_H_1,bdf_flux_err_dered_Ks_1,psf_flux_dered_calib_u_1,psf_flux_dered_calib_g_1,psf_flux_dered_calib_r_1,psf_flux_dered_calib_i_1,psf_flux_dered_calib_z_1,psf_flux_dered_calib_J_1,psf_flux_dered_calib_H_1,psf_flux_dered_calib_Ks_1,psf_flux_err_dered_calib_u_1,psf_flux_err_dered_calib_g_1,psf_flux_err_dered_calib_r_1,psf_flux_err_dered_calib_i_1,psf_flux_err_dered_calib_z_1,psf_flux_err_dered_calib_J_1,psf_flux_err_dered_calib_H_1,psf_flux_err_dered_calib_Ks_1,bdf_flux_dered_calib_u_1,bdf_flux_dered_calib_g_1,bdf_flux_dered_calib_r_1,bdf_flux_dered_calib_i_1,bdf_flux_dered_calib_z_1,bdf_flux_dered_calib_J_1,bdf_flux_dered_calib_H_1,bdf_flux_dered_calib_Ks_1,bdf_flux_err_dered_calib_u_1,bdf_flux_err_dered_calib_g_1,bdf_flux_err_dered_calib_r_1,bdf_flux_err_dered_calib_i_1,bdf_flux_err_dered_calib_z_1,bdf_flux_err_dered_calib_J_1,bdf_flux_err_dered_calib_H_1,bdf_flux_err_dered_calib_Ks_1,psf_mag_u_1,psf_mag_g_1,psf_mag_r_1,psf_mag_i_1,psf_mag_z_1,psf_mag_J_1,psf_mag_H_1,psf_mag_Ks_1,psf_mag_dered_u_1,psf_mag_dered_g_1,psf_mag_dered_r_1,psf_mag_dered_i_1,psf_mag_dered_z_1,psf_mag_dered_J_1,psf_mag_dered_H_1,psf_mag_dered_Ks_1,psf_mag_err_dered_u_1,psf_mag_err_dered_g_1,psf_mag_err_dered_r_1,psf_mag_err_dered_i_1,psf_mag_err_dered_z_1,psf_mag_err_dered_J_1,psf_mag_err_dered_H_1,psf_mag_err_dered_Ks_1,psf_mag_dered_calib_u_1,psf_mag_dered_calib_g_1,psf_mag_dered_calib_r_1,psf_mag_dered_calib_i_1,psf_mag_dered_calib_z_1,psf_mag_dered_calib_J_1,psf_mag_dered_calib_H_1,psf_mag_dered_calib_Ks_1,bdf_mag_u_1,bdf_mag_g_1,bdf_mag_r_1,bdf_mag_i_1,bdf_mag_z_1,bdf_mag_J_1,bdf_mag_H_1,bdf_mag_Ks_1,bdf_mag_dered_u_1,bdf_mag_dered_g_1,bdf_mag_dered_r_1,bdf_mag_dered_i_1,